In [11]:
import json
import re

poem = "nymphs_reply"
poem_path = f"../static/poems/{poem}/fulltext.txt"
source_poem = open(poem_path, "r").read()
stanza_no = 2
source_mp3 = f"../static/poems/{poem}/stanza{stanza_no}.mp3"
source_wav = f"../static/poems/{poem}/stanza{stanza_no}.wav"
MODEL_PATH = "./vosk-model-small-en-us-0.15"
source_transcript = source_poem.split("\n\n")[stanza_no - 1].replace("’", "")
grammar_words = re.findall(r'\w+', source_transcript.lower()) # + ["[unk]"]
# grammar_str = json.dumps(grammar_words)
grammar_str = json.dumps([" ".join(grammar_words)])
source_transcript
grammar_str

'["time drives the flocks from field to fold when rivers rage and rocks grow cold and philomel becometh dumb the rest complains of cares to come"]'

In [12]:
import vosk
import wave
import json

timestamps = []


wf = wave.open(source_wav, "rb")
model = vosk.Model(MODEL_PATH)
recognizer = vosk.KaldiRecognizer(model, wf.getframerate())
recognizer.SetWords(True)
recognizer.SetGrammar(grammar_str)

while True:
    data = wf.readframes(4000)
    if recognizer.AcceptWaveform(data):
        result = json.loads(recognizer.Result())
        words = result.get("result", [])
        timestamps += words
    else:
        pass
    if len(data) == 0:
        break
# finalize the recognizer
result = recognizer.FinalResult()
result = json.loads(result)
words = result.get("result", [])
timestamps += words
print(timestamps)
for i, timestamp in enumerate(timestamps):
    print(i, timestamp["word"], timestamp["start"], timestamp["end"])

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from ./vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from ./vosk-model-small-en-us-0.15/graph/HCLr.fst ./vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo ./vosk-model-small-en-us-0.15/graph/phones/word_boundary.int
LOG (VoskAPI:UpdateGrammarFst():recognizer.cc:287) ["time drives the flocks from field to fo

[{'conf': 1.0, 'end': 0.72, 'start': 0.36, 'word': 'time'}, {'conf': 1.0, 'end': 1.17, 'start': 0.72, 'word': 'drives'}, {'conf': 1.0, 'end': 1.29, 'start': 1.17, 'word': 'the'}, {'conf': 1.0, 'end': 1.77, 'start': 1.29, 'word': 'flocks'}, {'conf': 1.0, 'end': 1.95, 'start': 1.77, 'word': 'from'}, {'conf': 1.0, 'end': 2.31, 'start': 1.95, 'word': 'field'}, {'conf': 1.0, 'end': 2.49, 'start': 2.37, 'word': 'to'}, {'conf': 1.0, 'end': 3.03, 'start': 2.49, 'word': 'fold'}, {'conf': 1.0, 'end': 3.21, 'start': 3.03, 'word': 'when'}, {'conf': 1.0, 'end': 3.51, 'start': 3.21, 'word': 'rivers'}, {'conf': 1.0, 'end': 3.93, 'start': 3.51, 'word': 'rage'}, {'conf': 1.0, 'end': 4.02, 'start': 3.93, 'word': 'and'}, {'conf': 1.0, 'end': 4.41, 'start': 4.02, 'word': 'rocks'}, {'conf': 1.0, 'end': 4.59, 'start': 4.41, 'word': 'grow'}, {'conf': 1.0, 'end': 5.19, 'start': 4.59, 'word': 'cold'}, {'conf': 1.0, 'end': 5.85, 'start': 5.58, 'word': 'and'}, {'conf': 1.0, 'end': 6.492697, 'start': 5.88, 'word'

In [13]:
timestamps2 = [entry.copy() for entry in timestamps]
timestamps2[16]["word"] = "philomel"
timestamps2[17]["word"] = "becometh"
timestamps2.pop(18)
# timestamps2[5]["word"] = "trunkless"
# timestamps2[5]["end"] = timestamps2[7]["end"]
# timestamps2.pop(6)
# timestamps2.pop(6)
# timestamps2 = timestamps2[:10] + [{"word": "a", "start": 4.31, "end": 4.40, "conf": 1.0}] + timestamps2[10:]
# timestamps2.pop(21)
# timestamps2[-1]["end"] += 0.25
len(timestamps2)

26

In [14]:
outfile = f"../static/poems/{poem}/stanza{stanza_no}_timestamps.json"
with open(outfile, "w") as f:
    json.dump(timestamps2, f, indent=4)